In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# --- 1. Load TFLite Model and Tokenizer ---
print("Loading TFLite model and tokenizer...")
interpreter = tf.lite.Interpreter(model_path="/home/kronbii/Downloads/toxic_bert_dynamic.tflite")
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded.")

In [ ]:
# --- 2. Configure TFLite Interpreter for Single Sentence Inference ---
# We will process one sentence at a time, so we set the input shape to [1, 128] once.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get the index for each input tensor by name
input_ids_index = next(i['index'] for i in input_details if 'input_ids' in i['name'])
attention_mask_index = next(i['index'] for i in input_details if 'attention_mask' in i['name'])

# Define the fixed input shape [batch_size=1, sequence_length=128]
input_shape = [1, 128]

# Resize both input tensors for a single sentence
interpreter.resize_tensor_input(input_ids_index, input_shape)
interpreter.resize_tensor_input(attention_mask_index, input_shape)

# Allocate tensors ONCE before the loop
interpreter.allocate_tensors()
print("\n✅ TFLite interpreter configured for single sentence inference (shape [1, 128]).")

In [ ]:
# --- 3. Load and Prepare Dataset ---
print("\nLoading and cleaning dataset...")
try:
    input_df = pd.read_csv("/home/kronbii/repos/content-violence-detection/datasets/text/jigsaw/test.csv")
    gt_df = pd.read_csv("/home/kronbii/repos/content-violence-detection/datasets/text/jigsaw/test_labels.csv")
except FileNotFoundError:
    print("Error: Dataset files not found. Please update the file paths.")
    exit()

# Merge on 'id' to align text with labels
data = input_df.merge(gt_df, on="id", how="inner")

# Define the label columns
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Remove rows where any label is -1 (samples not used for scoring)
clean_data = data[~(data[label_cols] == -1).any(axis=1)].reset_index(drop=True)

print(f"Rows before cleaning: {len(data)}")
print(f"Rows after removing rows with -1 labels: {len(clean_data)}")
print("Dataset ready.")

In [ ]:
# --- 4. Run Sentence-by-Sentence Inference ---
print("\nRunning inference sentence by sentence...")

# Store predictions
all_preds = []
texts = clean_data['comment_text'].tolist()
ids = clean_data['id'].tolist()

# Loop through each sentence individually
for sentence in tqdm(texts, desc="Processing Sentences"):
    # Tokenize the single sentence
    inputs = tokenizer(
        sentence,
        return_tensors="tf", # Return TensorFlow tensors
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Set the input tensor values
    interpreter.set_tensor(input_ids_index, inputs['input_ids'])
    interpreter.set_tensor(attention_mask_index, inputs['attention_mask'])

    # Run inference
    interpreter.invoke()

    # Get the output, apply sigmoid
    output_data = interpreter.get_tensor(output_details[0]['index'])
    # The output shape is (1, 6), so we get the first element for our 1D probability array
    probs = tf.nn.sigmoid(output_data).numpy()[0]
    all_preds.append(probs)

# Combine the list of 1D arrays into a 2D numpy array
all_preds = np.array(all_preds)

# Build the output DataFrame
pred_df = pd.DataFrame(all_preds, columns=label_cols)
pred_df.insert(0, 'id', ids)

# Save predictions to a CSV file
pred_df.to_csv("tflite_text_jigsaw.csv", index=False)
print("TFLite predictions saved to tflite_predictions_sentence_by_sentence.csv")

In [ ]:
# --- 5. Evaluate Performance ---
print("\nEvaluating performance...")

# Get the ground truth labels
y_true = clean_data[label_cols].values
# Get the predicted probabilities
y_pred_probs = pred_df[label_cols].values

# Binarize predictions using a 0.5 threshold
y_pred = (y_pred_probs >= 0.5).astype(int)

print("\n🔍 Per-label Evaluation:\n")
for i, label in enumerate(label_cols):
    true = y_true[:, i]
    pred = y_pred[:, i]

    acc = accuracy_score(true, pred)
    f1 = f1_score(true, pred, average='binary', zero_division=0)

    print(f"Label: {label.capitalize()}")
    print(f"  Accuracy:          {acc:.4f}")
    print(f"  F1 Score:          {f1:.4f}")
    print("-" * 40)